In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.3/785.3 kB 11.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch==1.10.2 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0)
ERROR: No matching distribution found for torch==1.10.2


In [7]:
!wget -nc -O dataset/roc_stories/train2017.csv https://docs.google.com/spreadsheets/d/1emH8KL8NVCCumZc2oMu-3YqRWddD3AqZEHvNqMdfgKA/export?format=csv
!wget -nc -O dataset/roc_stories/valid2018.csv https://docs.google.com/spreadsheets/d/1F9vtluzD3kZOn7ULKyMQZfoRnSRzRnnaePyswkRqIdY/export?format=csv
!wget -nc -O dataset/roc_stories/test2016.csv  https://docs.google.com/spreadsheets/d/11tfmMQeifqP-Elh74gi2NELp0rx9JMMjnQ_oyGKqCEg/export?format=csv

--2023-05-06 00:13:15--  https://docs.google.com/spreadsheets/d/1emH8KL8NVCCumZc2oMu-3YqRWddD3AqZEHvNqMdfgKA/export?format=csv
Resolving docs.google.com (docs.google.com)... 74.125.124.138, 74.125.124.139, 74.125.124.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.124.138|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-0g-b8-sheets.googleusercontent.com/export/n4743ueaoiib6is67cgugk097k/8ikvvpuubt8btj1np8sou3q864/1683331995000/101032065670441238278/*/1emH8KL8NVCCumZc2oMu-3YqRWddD3AqZEHvNqMdfgKA?format=csv [following]
--2023-05-06 00:13:15--  https://doc-0g-b8-sheets.googleusercontent.com/export/n4743ueaoiib6is67cgugk097k/8ikvvpuubt8btj1np8sou3q864/1683331995000/101032065670441238278/*/1emH8KL8NVCCumZc2oMu-3YqRWddD3AqZEHvNqMdfgKA?format=csv
Resolving doc-0g-b8-sheets.googleusercontent.com (doc-0g-b8-sheets.googleusercontent.com)... 64.233.191.132, 2607:f8b0:4001:c0c::84
Connecting to doc-0g-b8-sheets.googleuserconte

In [8]:
!python prepare_stories.py --data_path dataset/roc_stories

Total data size is :  56107
Training data size is :  44885
Validation data size is :  5611
Testing data size is :  5611


In [9]:
!python -m nltk.downloader all
!pip install transformers
!pip install tensorrt

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque

In [12]:
!python prepare_keyword_extraction.py --data_path dataset/roc_stories/ --num_keywords $2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Process dataset/roc_stories/train.json
100% 44885/44885 [00:21<00:00, 2084.75it/s]
100% 44885/44885 [00:05<00:00, 8817.43it/s] 
Path:dataset/roc_stories/train.json, Data size:44885
Process dataset/roc_stories/val.json
100% 5611/5611 [00:02<00:00, 2234.32it/s]
100% 5611/5611 [00:00<00:00, 5994.64it/s]
Path:dataset/roc_stories/val.json, Data size:5611
Process dataset/roc_stories/test.json
100% 5611/5611 [00:03<00:00, 1460.66it/s]
100% 5611/5611 [00:00<00:00, 9416.22it/s]
Path:dataset/roc_stories/test.json, Data size:5611


In [13]:
!python run_conditional_modeling.py \
    --model_name_or_path gpt2 \
    --model_type dec \
    --do_train --do_eval \
    --train_data_file dataset/roc_stories/train.json \
    --eval_data_file dataset/roc_stories/val.json \
    --output_dir ./model/fine_tuned_models/stories_gpt_lsep5_psep10_e10_lr2e-5_b32_seed0 \
    --num_train_epochs 5 --learning_rate 2e-5 --new_learning_rate 1e-3 --seed 0 \
    --target_block_size 80 \
    --per_device_train_batch_size 8 --per_device_eval_batch_size 8 --total_batch_size 32 \
    --use_length --use_keyword --use_keyword_pos \
    --length_sep_num 5 --position_sep_num 10 --label_smoothing 0.1 \
    --keyword_pos_use_prob 0.90 --length_use_prob 0.90

2023-05-06 00:21:01.664185: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
05/06/2023 00:21:02 - WARNING - __main__ -   Local Rank : -1, with Device : cuda, n_gpu: 1, isDistributed: False, Training for 16bit: False
05/06/2023 00:21:02 - INFO - __main__ -   eval/train params are : Namespace(train_data_file='dataset/roc_stories/train.json', eval_data_file='dataset/roc_stories/val.json', test_data_file=None, min_sentence_size=-1, source_block_size=1024, target_block_size=80, local_rank=-1, output_dir='./model/fine_tuned_models/stories_gpt_lsep5_psep10_e10_lr2e-5_b32_seed0', do_train=True, do_eval=True, do_generate=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_device_generate_batch_size=8, total_batch_size=32, per_device_pretrain_batch_size=8, gradient_accumulation_steps=1, model_scratch=False, cache_dir=None, init_word_embedding=False, learning_rate=2e-05, new_learning_rate=0.001, weight_decay=0.01, adam_beta1=0.9,

In [14]:
!python run_conditional_modeling.py \
    --model_name_or_path ./model/fine_tuned_models/stories_gpt_lsep5_psep10_e10_lr2e-5_b32_seed0 \
    --model_type dec \
    --do_generate \
    --eval_data_file dataset/roc_stories/test.json \
    --output_dir out/stories_gpt_lsep5_psep10_e30_lr2e-5_b32_seed0 \
    --dataset_type stories \
    --per_device_generate_batch_size 1 \
    --use_length --use_keyword --use_keyword_pos \
    --length_sep_num 5 --position_sep_num 10 --temperature 0.1 --generation_method sample \
    --seed 0 --input_all_keyword

2023-05-06 01:55:13.871933: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
05/06/2023 01:55:15 - WARNING - __main__ -   Local Rank : -1, with Device : cuda, n_gpu: 1, isDistributed: False, Training for 16bit: False
05/06/2023 01:55:15 - INFO - __main__ -   eval/train params are : Namespace(train_data_file=None, eval_data_file='dataset/roc_stories/test.json', test_data_file=None, min_sentence_size=-1, source_block_size=1024, target_block_size=128, local_rank=-1, output_dir='out/stories_gpt_lsep5_psep10_e30_lr2e-5_b32_seed0', do_train=False, do_eval=False, do_generate=True, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_device_generate_batch_size=1, total_batch_size=256, per_device_pretrain_batch_size=8, gradient_accumulation_steps=1, model_scratch=False, cache_dir=None, init_word_embedding=False, learning_rate=0.0001, new_learning_rate=0.0001, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_g

In [16]:
!python eval_keyword_correct.py \
--data_path out/stories_gpt_lsep5_psep10_e30_lr2e-5_b32_seed0

out/stories_gpt_lsep5_psep10_e30_lr2e-5_b32_seed0
process dataset
5611it [00:06, 908.02it/s] 
list size: 5611
all keyword find 0.14471573694528606
all keyword position correct 0.010515059704152558


In [17]:
!python eval_generated_texts.py \
--data_path out/stories_gpt_lsep5_psep10_e30_lr2e-5_b32_seed0

process dataset
5611it [00:06, 918.76it/s] 
------------------sample-------------------------
0it [00:00, ?it/s]
------------------length control-------------------------
r-h length diff 13.534129388700766
r & h length average 48.662270540010695 37.13705221885582
0 1019
5 1191
10 1053
15 928
20 705
25 395
30 207
35 72
40 22
45 2
50 4
55 2
60 3
65 3
70 2
75 3
length diff 0:  10.05
length diff 1-5:  21.26
length diff 6-10:  20.03
length diff 11-:  48.65

----------------- keyword inclusion-----------------------------
r & h keyword find 1.0 0.3820174656923899
------------------position control------------------------------
r_pos & h_pos average 22.073694528604527 8.079076277116865
r-h pos diff 13.906228131560532
r-h relative pos diff 27.593124174552077
s-h relative pos diff 29.294354343870754
s-r relative pos diff 6.612026466136816
position: -1	6935	0.6179825343076101
position: 0	1915	0.17064694350383175
position: 10	637	0.056763500267332026
position: 20	409	0.03644626626269827
position: